In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import country_converter as coco
import numpy as np


In [2]:
summer_events = pd.read_csv('../Project-1/summer.csv')
winter_events = pd.read_csv('../Project-1/winter.csv')
country_vitals = pd.read_csv('../Project-1/dictionary.csv')


In [3]:
track_field = summer_events.loc[(summer_events['Sport'] == 'Athletics') & (summer_events['Year'] >= 1948)]
track_field_df = pd.DataFrame(track_field)
(track_field_df

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
7295,1948,London,Athletics,Athletics,"ALBERTSSON, Bertil",SWE,Men,10000M,Bronze
7296,1948,London,Athletics,Athletics,"ZATOPEK, Emil",TCH,Men,10000M,Gold
7297,1948,London,Athletics,Athletics,"MIMOUN, Alain",FRA,Men,10000M,Silver
7298,1948,London,Athletics,Athletics,"SCHWAB, Erich Arthur Fritz",SUI,Men,10000M Walk,Bronze
7299,1948,London,Athletics,Athletics,"MIKAELSSON, John",SWE,Men,10000M Walk,Gold
...,...,...,...,...,...,...,...,...,...
29767,2012,London,Athletics,Athletics,"CLAYE, Will",USA,Men,Triple Jump,Silver
29768,2012,London,Athletics,Athletics,"DONATO, Fabrizio",ITA,Men,Triple Jump,Bronze
29769,2012,London,Athletics,Athletics,"RYPAKOVA, Olga",KAZ,Women,Triple Jump,Gold
29770,2012,London,Athletics,Athletics,"IBARGUEN, Caterine",COL,Women,Triple Jump,Silver


In [8]:
track_field_df = track_field_df.drop(track_field_df[track_field_df['Event'] == '4X100M Relay'].index)
track_field_df = track_field_df.drop(track_field_df[track_field_df['Event'] == '4X400M Relay'].index)
track_field_df




,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
7295,1948,London,Athletics,Athletics,"ALBERTSSON, Bertil",SWE,Men,10000M,Bronze
7296,1948,London,Athletics,Athletics,"ZATOPEK, Emil",TCH,Men,10000M,Gold
7297,1948,London,Athletics,Athletics,"MIMOUN, Alain",FRA,Men,10000M,Silver
7298,1948,London,Athletics,Athletics,"SCHWAB, Erich Arthur Fritz",SUI,Men,10000M Walk,Bronze
7299,1948,London,Athletics,Athletics,"MIKAELSSON, John",SWE,Men,10000M Walk,Gold
...,...,...,...,...,...,...,...,...,...
29767,2012,London,Athletics,Athletics,"CLAYE, Will",USA,Men,Triple Jump,Silver
29768,2012,London,Athletics,Athletics,"DONATO, Fabrizio",ITA,Men,Triple Jump,Bronze
29769,2012,London,Athletics,Athletics,"RYPAKOVA, Olga",KAZ,Women,Triple Jump,Gold
29770,2012,London,Athletics,Athletics,"IBARGUEN, Caterine",COL,Women,Triple Jump,Silver


In [10]:
coco.CountryConverter().valid_class

['APEC',
 'BASIC',
 'BRIC',
 'CIS',
 'Cecilia2050',
 'DACcode',
 'EEA',
 'EU',
 'EU12',
 'EU15',
 'EU25',
 'EU27',
 'EU27_2007',
 'EU28',
 'EURO',
 'EXIO1',
 'EXIO1_3L',
 'EXIO2',
 'EXIO2_3L',
 'EXIO3',
 'EXIO3_3L',
 'Eora',
 'FAOcode',
 'G20',
 'G7',
 'GBDcode',
 'GWcode',
 'IEA',
 'IMAGE',
 'ISO2',
 'ISO3',
 'ISOnumeric',
 'MESSAGE',
 'OECD',
 'REMIND',
 'Schengen',
 'UN',
 'UNcode',
 'UNmember',
 'UNregion',
 'WIOD',
 'ccTLD',
 'continent',
 'name_official',
 'name_short',
 'obsolete',
 'regex']

In [9]:
track_field_df.to_csv('../Project-1/track_field.csv')
track_field_df = pd.read_csv('../Project-1/track_field.csv')


